In [285]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.corpus import stopwords

In [286]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [287]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           50000 non-null  int64 
 1   review_es    50000 non-null  object
 2   sentimiento  50000 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


Aqui ya observamos que el dataset no contiene nulos

In [288]:
df_train.head(10)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,positivo
1,1,Una pequeña pequeña producción.La técnica de f...,positivo
2,2,Pensé que esta era una manera maravillosa de p...,positivo
3,3,"Básicamente, hay una familia donde un niño peq...",negativo
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",positivo
5,5,Probablemente mi película favorita de todos lo...,positivo
6,6,Seguro que me gustaría ver una resurrección de...,positivo
7,7,"Este espectáculo fue una idea increíble, fresc...",negativo
8,8,Alentados por los comentarios positivos sobre ...,negativo
9,9,"Si te gusta la risa original desgarradora, te ...",positivo


In [289]:
df_test.head(3)

,ID,review_es
0,60000,La mayor virtud de esta película es su existen...
1,60001,"No soy un experto cinéfilo, pero pocas veces m..."
2,60002,Si no eres un incondicional del humor estilo T...


In [290]:
x_train, y_train = df_train["review_es"], df_train["sentimiento"]

In [291]:
x_test = df_test["review_es"]

In [292]:
x_train, y_train

(0        Uno de los otros críticos ha mencionado que de...
 1        Una pequeña pequeña producción.La técnica de f...
 2        Pensé que esta era una manera maravillosa de p...
 3        Básicamente, hay una familia donde un niño peq...
 4        El "amor en el tiempo" de Petter Mattei es una...
                                ...                        
 49995    Pensé que esta película hizo un buen trabajo a...
 49996    Mala parcela, mal diálogo, mala actuación, dir...
 49997    Soy católica enseñada en escuelas primarias pa...
 49998    Voy a tener que estar en desacuerdo con el com...
 49999    Nadie espera que las películas de Star Trek se...
 Name: review_es, Length: 50000, dtype: object,
 0        positivo
 1        positivo
 2        positivo
 3        negativo
 4        positivo
            ...   
 49995    positivo
 49996    negativo
 49997    negativo
 49998    negativo
 49999    negativo
 Name: sentimiento, Length: 50000, dtype: object)

In [293]:
from sklearn.model_selection import GridSearchCV

In [294]:
multinomial_classifier = MultinomialNB()

In [295]:
vectorizer = CountVectorizer()
x_train_vec = vectorizer.fit_transform(x_train)

In [296]:
param_grid = {
    'alpha': np.arange(1, 10.1, 0.1),
    'fit_prior': [True, False]
}

grid_search = GridSearchCV(estimator=multinomial_classifier, param_grid=param_grid, cv=5, scoring='accuracy')

grid_search.fit(x_train_vec, y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(),
             param_grid={'alpha': array([ 1. ,  1.1,  1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,
        2.1,  2.2,  2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,
        3.2,  3.3,  3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,
        4.3,  4.4,  4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,
        5.4,  5.5,  5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,
        6.5,  6.6,  6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,
        7.6,  7.7,  7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,
        8.7,  8.8,  8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,
        9.8,  9.9, 10. ]),
                         'fit_prior': [True, False]},
             scoring='accuracy')

In [297]:
grid_search.best_params_

{'alpha': 1.1, 'fit_prior': True}

In [298]:
best_mnb = MultinomialNB(alpha=grid_search.best_params_['alpha'], fit_prior=grid_search.best_params_['fit_prior'])

In [299]:
x_test_vec = vectorizer.transform(x_test)

best_mnb.fit(x_train_vec, y_train)

y_pred = best_mnb.predict(x_test_vec)

In [300]:
df_predicciones = pd.DataFrame({"ID": df_test["ID"], "prediccion": y_pred})

df_predicciones.to_csv("predicciones.csv", header=["ID", "sentimiento"], index=False)

In [301]:
df_predicciones

,ID,prediccion
0,60000,negativo
1,60001,negativo
2,60002,negativo
3,60003,negativo
4,60004,negativo
...,...,...
8594,68594,positivo
8595,68595,negativo
8596,68596,positivo
8597,68597,negativo


Pruebo sacando los stopwords

In [302]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/taiel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [303]:
stop_words = set(stopwords.words("spanish"))

In [304]:
#stop_words

In [305]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/taiel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [306]:
def limpiar_reseña(reseña):
    palabras = word_tokenize(reseña.lower())
    palabras_filtradas = [word for word in palabras if word not in stop_words]
    return " ".join(palabras_filtradas)

In [307]:
# x_train_ = [limpiar_reseña(reseña) for reseña in x_train]

In [308]:
# x_train_ = pd.Series(x_train_)

In [309]:
# x_train_

In [310]:
# vectorizer = CountVectorizer()
# x_train_vec_ = vectorizer.fit_transform(x_train_)
# x_test_vec_ = vectorizer.transform(x_test)

# nb_classifier = MultinomialNB()
# nb_classifier.fit(x_train_vec_, y_train)

# y_pred = nb_classifier.predict(x_test_vec_)

In [311]:
# df_predicciones = pd.DataFrame({"ID": df_test["ID"], "prediccion": y_pred})

# df_predicciones.to_csv("predicciones.csv", header=["ID", "sentimiento"], index=False)